In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [ ]:
### colab 사용 시 실행
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir('gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/src/ipynb')

In [ ]:
os.chdir('./../../')
from src.basic_utils import *
from src.modeling_utils import *

import re
from khaiii import KhaiiiApi

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
val = pd.read_json(data_path+'val(lower).json')
test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )    # 연도+월
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))                        # 연도
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
test['date'] = test['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
test['year'] = test['updt_date'].apply(lambda x: int(x[2:4]))

all = pd.concat([train, val, test])
date_list = sorted(list(all['date'].unique()))

date_dict = {}       # 해당 연도+월 에 index 부여(?)
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)

In [ ]:
### khaiii 활용한 plylst_title 형태소 분석 통한 태그 생성
val = add_tags(val)
test = add_tags(test)

In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)
val2 = val[val.index.isin(no_song)]
val4 = val[val.index.isin(no_both)]

no_tag, no_song, yes_index, no_both = check_target_type(test)
test2 = test[test.index.isin(no_song)]
test4 = test[test.index.isin(no_both)]

In [ ]:
val2_predict = pred_v2(val2, tag_song_dict, song_tag_dict, popular_date_song, popular_date_tag)
write_json(val2_predict, data_path+'results/val/val2_predict_withTitle.json')

val4_predict = pred_v4(val4, tag_song_dict, song_tag_dict, popular_date_song, popular_date_tag)
write_json(val4_predict, data_path+'results/val/val4_predict_withTitle.json', 'utf-16')

In [ ]:
test2_predict = pred_v2(test2, tag_song_dict, song_tag_dict, popular_date_song, popular_date_tag)
write_json(test2_predict, data_path+'results/test/test2_predict_withTitle.json')
test4_predict = pred_v4(test4, tag_song_dict, song_tag_dict, popular_date_song, popular_date_tag)
write_json(test4_predict, data_path+'results/test/test4_predict_withTitle.json')